## Convert image sequences, based on spreadsheet entries

In [2]:
# import csv
import pandas as pd
import numpy as np
import os
# import videoTools as vt

# Change to for each computer
root_path = '/Users/mmchenry/Documents/Projects/geotaxis'

# Subdirectories
csv_path  = root_path + '/Seastar geotaxis experiment log - Experiments.csv'
im_path   = root_path + '/Raw_images'
out_path  = root_path + '/Videos'

# Frame rate, image quality, image suffix
fps = 1
imQuality = 0.75
suffix = "JPG"

# Quality value, on the 51-point scale used by ffmpeg
qVal = 51 * (1 - imQuality)

# Open CSV file
file = open(csv_path)

# Import CSV data
data = pd.read_csv(file)

# Number of videos to analyze
nVids = np.nansum(data.make_video)

# COunter for number of videos analyzed
k = 1

# Loop though each row 
for n in range(len(data)):

    if data.make_video[n]==1:

        print('Converting ' + str(data.date[n]) + '_' + str(int(data.trial_num[n])) + ' . . .')

        # Read number of frames from spreadsheet
        frStart = int(data.start_image_filename[n][3:])
        frEnd   = int(data.end_image_filename[n][3:])
        nFrames = frEnd - frStart

        vidPath = out_path + '/' + str(data.date[n]) + '_' + str(int(data.trial_num[n])) + '.mp4'
        imagePath = im_path + '/' + str(data.date[n]) + '/'

        command = f"ffmpeg -start_number {frStart}  -loglevel warning -i {imagePath}/DSC%05d.{suffix} \
                 -vframes {nFrames} -y -vcodec libx264 -pix_fmt yuv420p -an -r {fps} -crf {qVal}  '{vidPath}'"
        os.system(command)

        print('     Completed ' + str(int(k)) + ' of ' + str(int(nVids)) + ' videos')
        # print('Completed videos' + )
        k = k + 1

# Close CSV
file.close()


Converting 2022-02-16_2 . . .


[swscaler @ 0x1489e0000] [swscaler @ 0x1489f0000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x138028000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x138038000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x138058000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x138090000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x1380a0000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x138198000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x1381a8000] deprecated pixel format used, make sure you did set range correctly
[swscaler @ 0x1489e0000] [swscaler @ 0x138188000] deprec

     Completed 1 of 1 videos
